In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import math as math

In [3]:
cur=conn.cursor()

In [4]:
#removed some AND clauses from cur.execute, issleeve isdeck
# COALESCE(cardquantity,maybecardquantity,1)
cur.execute('''SELECT cardname,carddate,AVG(pshipq) as AVGE,percentile_cont(0.5) WITHIN GROUP (ORDER BY pshipq) as MED,COUNT(*) as VLM 
    FROM (SELECT DISTINCT title,cardname,carddate,(price+shipping)/COALESCE(cardquantity,maybecardquantity,1) as pshipq FROM public.transactions
	WHERE possiblybad IS FALSE 
    AND isboosterbox IS NULL
	AND cardname IN(SELECT DISTINCT(cardname) FROM public.tourninfo)
	AND isfoil IS FALSE
    AND cardlanguage IN('english',NULL)
    AND cardset NOT IN('LEA','LEB','U')
    AND isemblem IS NULL
	AND cardspecial IS NULL
	AND isplaymat IS NULL
    AND isgraded IS NULL
    AND iscommanderdeck IS NULL
    AND isultra IS NULL
    AND (lotis IS FALSE OR lotis IS NULL)
    AND issleeve IS NULL
    AND isdeck IS NULL
    AND iscommanderdeck IS NULL
    AND istoken IS NULL
	AND (othercards IS NULL OR othercards='Hour of Devastation;' OR transplit IS TRUE)
	AND saletype IN('normal','italic')) as foo GROUP BY carddate,cardname ORDER BY cardname,carddate ASC''')
# ,'italic'

In [5]:
rows=cur.fetchall()
cur.close()
conn.close()

In [6]:
df=pd.DataFrame(rows,columns=["cardname","carddate","avg","med","quant"])
df['carddate'] = df['carddate'].astype('datetime64[ns]') 

In [7]:
overalllist=[]
for cardname in df["cardname"].unique():
    coeflist=[]
    regvals=df.loc[df['cardname'] == cardname]
    regvals["carddate"]=regvals["carddate"]-regvals.iloc[0]["carddate"]
    regvals["carddate"]=regvals["carddate"].astype('int64')/86400000000000
    for frac in [1-1/2**i for i in range(0,math.floor(math.log(len(regvals),2)))]:
#     for frac in [0,.5]:
        if len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])<8:
            pass
        else:
#             data[['flow_h','flow_c']].min(axis=1)
            minarr=np.minimum(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["avg"],regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["med"])
            regmin=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),minarr)
            regmed=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["med"])
            regavg=LinearRegression().fit(np.array(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["carddate"]).reshape(-1, 1),regvals.iloc[math.floor(frac*len(regvals)):len(regvals)]["avg"])
            medcoef=regmed.coef_[0]
            avgcoef=regavg.coef_[0]
            mincoef=regmin.coef_[0]
            coeflist.append((mincoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
            coeflist.append((medcoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
            coeflist.append((avgcoef,len(regvals.iloc[math.floor(frac*len(regvals)):len(regvals)])))
    coeflist.append(cardname)
    overalllist.append(coeflist[::-1])
    
# coefdf=pd.DataFrame(overalllist,columns=["cardname","mecoef1","avgcoef1","medcoef.5","avgcoef.5"])
coefdf=pd.DataFrame(overalllist)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [9]:
coefdf.dropna(thresh=2).sort_values(by=3,axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
872,"Urborg, Tomb of Yawgmoth","(-1.1795975530029055, 8)","(-1.1795975530029055, 8)","(-1.1795975530029055, 8)","(-0.01926771561355437, 15)","(-0.01926771561355437, 15)","(-0.01926771561355437, 15)","(0.06361488754818698, 29)","(0.0636241097165201, 29)","(0.0636241097165201, 29)",None,None,None,None,None,None
395,Hour of Devastation,"(-0.4980859354448818, 7)","(-0.49886718625202786, 7)","(-0.49886718625202786, 7)","(-0.044986013565746, 13)","(-0.04515264073146917, 13)","(-0.04515264073146917, 13)","(0.008527465095842457, 25)","(0.008490337866119751, 25)","(0.008490337866119751, 25)","(-0.004046767179836867, 49)","(-0.004061017149128239, 49)","(-0.004061017149128239, 49)","(0.002897032786341484, 97)","(0.0029304412139602456, 97)","(0.002983799439045599, 97)"
342,Godless Shrine,"(-0.36566669827415466, 6)","(-0.38328574044363634, 6)","(-0.38328574044363634, 6)","(-0.09445974588541274, 12)","(-0.09716857454391195, 12)","(-0.09716857454391195, 12)","(0.04382250770293969, 23)","(0.043790980259305654, 23)","(0.04504305294720114, 23)","(-0.004152168660372795, 45)","(-0.004068188695310878, 45)","(-0.004174801107729649, 45)","(-0.009701020838747704, 89)","(-0.009382533892252555, 89)","(-0.009452190936436903, 89)"
526,Monastery Mentor,"(-0.3275619932442665, 5)","(-0.3275619932442665, 5)","(-0.3275619932442665, 5)","(-0.05012646873807908, 10)","(-0.05012646873807908, 10)","(-0.05012646873807908, 10)","(-0.00606365003577948, 20)","(-0.00606365003577948, 20)","(-0.00606365003577948, 20)","(-0.04924712986144209, 40)","(-0.04915199288442575, 40)","(-0.04915199288442575, 40)",None,None,None
565,"Nykthos, Shrine to Nyx","(-0.3034350309914645, 5)","(-0.3034350309914645, 5)","(-0.3034350309914645, 5)","(-0.13824673147176442, 9)","(-0.13824673147176442, 9)","(-0.13824673147176442, 9)","(-0.07327862479233323, 17)","(-0.07169636773046206, 17)","(-0.07169636773046206, 17)","(0.020671835482038502, 34)","(0.020612266897972832, 34)","(0.020612266897972832, 34)","(0.05531986725861134, 68)","(0.05532038411473449, 68)","(0.05532038411473449, 68)"
803,Temple Garden,"(-0.27892842973981075, 6)","(-0.27892842973981075, 6)","(-0.27892842973981075, 6)","(0.027411520055290836, 12)","(0.024125758665571755, 12)","(0.024125758665571755, 12)","(0.013663453525653074, 24)","(0.009206454680300973, 24)","(0.009206454680300973, 24)","(0.02584341537621515, 48)","(0.025167223001161997, 48)","(0.025174454694288625, 48)","(0.01189289527994804, 96)","(0.011553030914586078, 96)","(0.011553344474421547, 96)"
310,"Gadwick, the Wizened","(-0.2707146626115985, 5)","(-0.2707146626115985, 5)","(-0.2707146626115985, 5)","(0.005913330936602181, 10)","(0.005913330936602181, 10)","(0.005913330936602181, 10)","(0.02744353554308241, 20)","(0.02744353554308241, 20)","(0.02744353554308241, 20)",None,None,None,None,None,None
335,Goblin Piledriver,"(-0.2204985316198246, 5)","(-0.2204985316198246, 5)","(-0.2204985316198246, 5)","(-0.009322427884585718, 9)","(-0.009322427884585718, 9)","(-0.009322427884585718, 9)","(-0.005687210741567058, 18)","(-0.005687210741567058, 18)","(-0.005687210741567058, 18)",None,None,None,None,None,None
80,Blood Crypt,"(-0.2107143674577958, 6)","(-0.2107143674577958, 6)","(-0.2107143674577958, 6)","(0.12205641732897238, 11)","(0.12205641732897238, 11)","(0.12205641732897238, 11)","(0.03189639289719769, 22)","(0.03186979455236569, 22)","(0.03189639289719769, 22)","(-0.01982061741739983, 44)","(-0.01954333075114101, 44)","(-0.019656465619951994, 44)","(0.01203869442197999, 88)","(0.012052036569141961, 88)","(0.012095410005441534, 88)"
483,"Liliana, the Last Hope","(-0.18071974520820422, 8)","(-0.18071974520820422, 8)","(-0.18071974520820422, 8)","(-0.06829010939815096, 15)","(-0.06829010939815096, 15)","(-0.06829010939815096, 15)","(-0.05330568023099379, 29)","(-0.05507376501517359, 29)","(-0.05330568023099379, 29)",None,None,None,None,None,None
